In [1]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
 
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")
noise_model = NoiseModel.from_backend(backend)

qiskit_runtime_service.__init__:INFO:2024-03-06 13:10:23,187: Default instance: ibm-q/open/main


In [3]:
# System Specification
n_qubits = 64
circ = QuantumCircuit(n_qubits)

# Apply Hadamard gate to all qubits
for qubit in range(n_qubits):
    circ.h(qubit)

# Measure all qubits
circ.measure_all()
# print(circ)

In [4]:
# Ideal simulator and execution
sim_ideal = AerSimulator()

# Number of sequences to generate
num_sequences = 2

# Number of times to run the circuit to get 512 bits
num_runs = 512 // n_qubits

# Open the file in write mode
with open('Ideal_seq.txt', 'w') as f:
    # Repeat for each sequence
    for _ in range(num_sequences):
        # Initialize an empty string to store the binary sequence
        binary_sequence = ''

        # Generate the sequence
        for _ in range(num_runs):
            result_ideal = sim_ideal.run(circ).result()
            counts = result_ideal.get_counts()  # number of times - unique binary sequences

            # Get the most frequent bitstring
            most_frequent_bitstring = max(counts, key=counts.get)
            binary_sequence += most_frequent_bitstring

        # Write the sequence to the file
        f.write(binary_sequence + '\n')


In [5]:
# Noise Example 1: Basic bit-flip error noise model
# Lets consider a simple toy noise model example common in quantum information theory research:

# When applying a single qubit gate, flip the state of the qubit with probability p_gate1.
# When applying a 2-qubit gate apply single-qubit errors to each qubit.
# When resetting a qubit reset to 1 instead of 0 with probability p_reset.
# When measuring a qubit, flip the state of the qubit with probability p_meas.

# Example error probabilities
p_reset = 0.03
p_meas = 0.1
p_gate1 = 0.05
 
# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)
 
# Add errors to noise model
noise_bit_flip = NoiseModel()
noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])
 
print(noise_bit_flip)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u1', 'u2', 'u3']
  Instructions with noise: ['u2', 'measure', 'reset', 'u1', 'cx', 'u3']
  All-qubits errors: ['reset', 'measure', 'u1', 'u2', 'u3', 'cx']


In [6]:
# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_bit_flip)
 
# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(optimization_level=3, backend=sim_noise)
circ_tnoise = passmanager.run(circ)

# Number of sequences to generate
num_sequences = 2

# Number of times to run the circuit to get 512 bits
num_runs = 512 // n_qubits

# Open the file in write mode
with open('Flip_bit_seq.txt', 'w') as f:
    # Repeat for each sequence
    for _ in range(num_sequences):
        # Initialize an empty string to store the binary sequence
        binary_sequence = ''

        # Generate the sequence
        for _ in range(num_runs):
            # Run and get counts
            result_bit_flip = sim_noise.run(circ_tnoise).result()
            counts_bit_flip = result_bit_flip.get_counts(0)

            # Get the most frequent bitstring
            most_frequent_bitstring = max(counts_bit_flip, key=counts_bit_flip.get)
            binary_sequence += most_frequent_bitstring

        # Write the sequence to the file
        f.write(binary_sequence + '\n')

Traceback (most recent call last):
  Cell In[6], line 6
    circ_tnoise = passmanager.run(circ)
  File /opt/conda/lib/python3.10/site-packages/qiskit/transpiler/passmanager.py:397 in run
    return super().run(circuits, output_name, callback, num_processes=num_processes)
  File /opt/conda/lib/python3.10/site-packages/qiskit/transpiler/passmanager.py:422 in wrapper
    raise TranspilerError(ex.message) from ex
TranspilerError: "HighLevelSynthesis was unable to synthesize Instruction(name='measure', num_qubits=1, num_clbits=1, params=[])."

Use %tb to get the full traceback.


In [7]:
# T1 and T2 values for qubits 0-3
n_bits = 4
T1s = np.random.normal(50e3, 10e3, n_bits) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(70e3, 10e3, n_bits)  # Sampled from normal distribution mean 50 microsec
 
# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(n_bits)])
 
# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond
 
# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]
 
# Add errors to noise model
noise_thermal = NoiseModel()
for j in range(n_bits):
    noise_thermal.add_quantum_error(errors_reset[j], "reset", [j])
    noise_thermal.add_quantum_error(errors_measure[j], "measure", [j])
    noise_thermal.add_quantum_error(errors_u1[j], "u1", [j])
    noise_thermal.add_quantum_error(errors_u2[j], "u2", [j])
    noise_thermal.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(n_bits):
        noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [j, k])
 
print(noise_thermal)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u2', 'u3']
  Instructions with noise: ['u2', 'measure', 'reset', 'cx', 'u3']
  Qubits with noise: [0, 1, 2, 3]
  Specific qubit errors: [('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('u2', (0,)), ('u2', (1,)), ('u2', (2,)), ('u2', (3,)), ('u3', (0,)), ('u3', (1,)), ('u3', (2,)), ('u3', (3,)), ('cx', (0, 0)), ('cx', (0, 1)), ('cx', (0, 2)), ('cx', (0, 3)), ('cx', (1, 0)), ('cx', (1, 1)), ('cx', (1, 2)), ('cx', (1, 3)), ('cx', (2, 0)), ('cx', (2, 1)), ('cx', (2, 2)), ('cx', (2, 3)), ('cx', (3, 0)), ('cx', (3, 1)), ('cx', (3, 2)), ('cx', (3, 3))]


In [9]:
# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_bit_flip)
 
# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(optimization_level=3, backend=sim_noise)
circ_tnoise = passmanager.run(circ)
 
# Initialize an empty string to store the binary sequence
binary_sequence = ''

# Number of times to run the circuit to get 512 bits
num_runs = 512 // n_qubits

for _ in range(num_runs):
    # Run and get counts
    result_thermal = sim_thermal.run(circ_tthermal).result()
    counts_thermal = result_thermal.get_counts(0)
    
    # Get the most frequent bitstring
    most_frequent_bitstring = max(counts_thermal, key=counts_thermal.get)
    binary_sequence += most_frequent_bitstring

print('RESULT: ', binary_sequence, '\n')
print(len(binary_sequence))



# =========================================================================================
# Number of sequences to generate
num_sequences = 2

# Number of times to run the circuit to get 512 bits
num_runs = 512 // n_qubits

# Open the file in write mode
with open('Thermal_seq.txt', 'w') as f:
    # Repeat for each sequence
    for _ in range(num_sequences):
        # Initialize an empty string to store the binary sequence
        binary_sequence = ''

        # Generate the sequence
        for _ in range(num_runs):
            # Run and get counts
            result_thermal = sim_thermal.run(circ_tthermal).result()
            counts_thermal = result_thermal.get_counts(0)

            # Get the most frequent bitstring
            most_frequent_bitstring = max(counts_thermal, key=counts_thermal.get)
            binary_sequence += most_frequent_bitstring

        # Write the sequence to the file
        f.write(binary_sequence + '\n')

Traceback (most recent call last):
  Cell In[9], line 6
    circ_tnoise = passmanager.run(circ)
  File /opt/conda/lib/python3.10/site-packages/qiskit/transpiler/passmanager.py:397 in run
    return super().run(circuits, output_name, callback, num_processes=num_processes)
  File /opt/conda/lib/python3.10/site-packages/qiskit/transpiler/passmanager.py:422 in wrapper
    raise TranspilerError(ex.message) from ex
TranspilerError: "HighLevelSynthesis was unable to synthesize Instruction(name='measure', num_qubits=1, num_clbits=1, params=[])."

Use %tb to get the full traceback.
